In [ ]:
import pandas as pd
import numpy as np
import os

# Cargar archivo CSV crudo
datosTocumenCrudos = pd.read_csv("../../data/raw/datos_tocumen_crudos_2024.csv")

# Función para convertir grados a 8 puntos cardinales
def gradosAPuntoCardinal(deg):
    directions = [
        'Norte', 'Noreste', 'Este', 'Sureste',
        'Sur', 'Suroeste', 'Oeste', 'Noroeste'
    ]
    ix = int((deg + 22.5) / 45) % 8
    return directions[ix]

# Renombrar columnas para que coincidan con el formato limpio
limpiezaTocumen = datosTocumenCrudos.rename(columns={
    "date": "Date",
    "tmax": "Tmax",
    "tmin": "Tmin",
    "tavg": "Tavg",
    "prcp": "Prcp",
    "wspd": "Wspd",
    "wdir": "Wdir"
})

# Convertir fechas a formato YYYY-MM-DD
limpiezaTocumen["Date"] = pd.to_datetime(limpiezaTocumen["Date"]).dt.date

# Convertir velocidad del viento de m/s a km/h
limpiezaTocumen["Wspd"] = (limpiezaTocumen["Wspd"] * 3.6).round(1)

# Simplificar dirección del viento
limpiezaTocumen["Wdir"] = limpiezaTocumen["Wdir"].apply(
    lambda x: gradosAPuntoCardinal(x) if not pd.isnull(x) else np.nan
)

# Agregar columnas fijas
limpiezaTocumen["Estacion"] = "Tocumen"
limpiezaTocumen["Provincia"] = "Panamá"

# Eliminar columnas irrelevantes
limpiezaTocumen.drop(columns=["snow", "wpgt", "pres", "tsun"], inplace=True)

# Reordenar columnas al formato deseado
limpiezaTocumen = limpiezaTocumen[[
    "Date", "Tmax", "Tmin", "Tavg", "Prcp", "Wspd", "Wdir", "Estacion", "Provincia"
]]

destinoArchivoLimpio = "../../data/cleaned/datos_limpios_tocumen_2024.csv"
os.makedirs(os.path.dirname(destinoArchivoLimpio), exist_ok=True)

# Guardar el archivo sin comillas, usando codificación UTF-8
limpiezaTocumen.to_csv(
    destinoArchivoLimpio,
    index=False,
    encoding="utf-8"
)

print(f"Archivo limpio guardado en: {destinoArchivoLimpio}")